# Library design for CTP02, 

## EMT single molecule FISH check
by Pu Zheng



In [1]:
#minimum imports:
import time,os,sys,glob
import cPickle as pickle
import numpy as np
import khmer
sys.path.append(r'/n/home13/pzheng/Documents/python-functions/python-functions-library')

from LibraryConstruction import fastaread,fastawrite,fastacombine
import LibraryDesigner as ld
import LibraryConstruction as lc

from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio.SeqRecord import SeqRecord 

## 1. Load previously designed probes, generate pb_records and pb_lists

In [2]:
# dir
master_dir = r'/n/boslfs/LABS/zhuang_lab/User/pzheng/Libraries/CTP-02/EMT_smFISH';
# input filename
pb_filename = 'EMT-smFISH.csv';

pb_records, gene_pb_dic = [],{}; # initialize

import csv
with open(master_dir+os.sep+pb_filename,'rU') as handle:
    _csvreader = csv.reader(handle, delimiter=',') # reader
    # check if has header
    _has_header = csv.Sniffer().has_header(handle.readline())
    handle.seek(0)  # Rewind.
    if _has_header:
        next(_csvreader)  # Skip header row.
    for row in _csvreader:
        _name = row[0];
        _seq = row[1].upper()
        pb_records.append(SeqRecord(Seq(_seq,alphabet=IUPAC.unambiguous_dna),id=_name, name=_name,description=''))
        #reg_id = int(name.split('reg_')[1].split("_")[0])
        gene_id = _name.split('gene_')[1].split("_")[0]
        pb_info = {'reg_index':gene_id, 'total_seq':_seq, 'total_name':_name};
        if gene_id not in gene_pb_dic.keys():
            gene_pb_dic[gene_id] = [pb_info];
        else:
            gene_pb_dic[gene_id].append(pb_info);
# define pb_lists, match format
pb_lists = gene_pb_dic.values()

# save
save_dir = 'final_probes'
if not os.path.exists(master_dir+os.sep+save_dir):
    os.makedirs(master_dir+os.sep+save_dir)
print "-- Save pb_lists"
pickle.dump(pb_lists, open(master_dir+os.sep+save_dir+os.sep+'list.pkl', 'w'));
print "-- Save pb_records"
with open(master_dir+os.sep+save_dir+os.sep+'candidate_probes.fasta', "w") as output_handle:
    SeqIO.write(pb_records, output_handle, 'fasta');

-- Save pb_lists
-- Save pb_records


### Read barcode Sequences

In [3]:
# read all Stv barcodes
barcode_dir = r'/n/boslfs/LABS/zhuang_lab/User/pzheng/Barcodes';

#stv_adaptor = [1,2,17,62,77,78,79,80,81,82,83,84] # barcodes saved for adaptors
#stv_bad = [34,38,41] # barcodes performed badly
#stv_mask = stv_adaptor + stv_bad 
stv_mask = []

with open(barcode_dir+os.sep+'top_Stvs.fasta', "rU") as handle:
    stv_barcodes = [];
    for record in SeqIO.parse(handle, "fasta"):
        if int(record.id.split('_')[1]) not in stv_mask:
            stv_barcodes.append(record);

# read all NDB barcodes
ndb_mask = [];

with open(barcode_dir+os.sep+'NDBs.fasta', "rU") as handle:
    ndb_barcodes = [];
    for record in SeqIO.parse(handle, "fasta"):
        if int(record.id.split('_')[1]) not in ndb_mask:
            ndb_barcodes.append(record);
print "Barcodes loaded: Stv: "+str(len(stv_barcodes))+", NDB: "+str(len(ndb_barcodes));

Barcodes loaded: Stv: 75, NDB: 1052


### Read PCR primers

In [4]:
primer_dir = r'/n/boslfs/LABS/zhuang_lab/User/pzheng/Primers';
fwd_primer_filename = 'forward_primers_keep.fasta';
rev_primer_filename = 'reverse_primers_keep.fasta';

# read all forward primers
with open(primer_dir+os.sep+fwd_primer_filename, "rU") as handle:
    fwd_primers = [];
    for record in SeqIO.parse(handle, "fasta"):
        fwd_primers.append(record);
# read all forward primers
with open(primer_dir+os.sep+rev_primer_filename, "rU") as handle:
    rev_primers = [];
    for record in SeqIO.parse(handle, "fasta"):
        rev_primers.append(record);
print "Primers loaded: forward: "+str(len(fwd_primers))+", reverse: "+str(len(rev_primers));    

# primers
fprimer = fwd_primers[4];
print '- forward primer:', fprimer
rprimer = rev_primers[3];
print '- reverse primer:', rprimer

Primers loaded: forward: 12, reverse: 9
- forward primer: ID: W1A09_primer_8
Name: W1A09_primer_8
Description: W1A09_primer_8
Number of features: 0
Seq('TTGTTGAGGCGGCGGAAGTC', SingleLetterAlphabet())
- reverse primer: ID: W1A08_primer_7
Name: W1A08_primer_7
Description: W1A08_primer_7
Number of features: 0
Seq('TAATACGACTCACTATAGGGCCCTTTCGCGCTGCATCGAC', SingleLetterAlphabet())


In [5]:
def Check_Probes(pb_records, pb_lists, master_dir, 
                 fwd_primer,rev_primer,
                 stv_barcodes, ndb_barcodes,
                 RNA=False,RNA_padding=6,
                 add_rand_gap=0, total_bc=3, barcode_len=20, 
                 target_len=42, word_size=17, 
                 max_internal_hits=5, max_genome_hits=200,
                 index_folder=r'/n/boslfs/LABS/zhuang_lab/User/pzheng/Indeces/human/hg38',
                 save_dir=r'final_probes', save=True, verbose=True):
    '''Functions to check probe quality
    Inputs:
        pb_records: probe information in biopython format, list of SeqRecords
        pb_lists: list of lists of probe information dictionary, list of list
        master_dir: directory of this sub library, string
        fwd_primer: forward primer in biopython format, SeqRecord
        rev_primer: reverse primer in biopython format, SeqRecord
        stv_barcodes: stv_barcodes in biopython format, list of SeqRecord
        ndb_barcodes: ndb_barcodes in biopython format, list of SeqRecord
        (optional)
        RNA: whether this library is for RNA or not, bool(false)
        add_rand_gap: number of random bases between barcodes, int(0)
        total_bc: number of barcodes per probe, int(3)
        barcode_len: length of barcode binding site on probe, int(20)
        target_len: length of targeting region on probe, int(42)
        word_size: word size for indeces and for probe designer, int(17)
        max_internal_hit: maximal internal k-mer allowed, int(5)
        max_genome_hits: maximal genome k-mer allowed, int(200)
        index_folder: full directory to indeces, string
        save_dir: sub directory for saving, under master_dir, string
        save: whether save, bool(True)
        verbose: say something during checking probes, bool(True)
    Output:
        kept_records: all seq records that passed filter
        _size_from_rec: size of each region/gene, dic
        '''
    # imports
    import os,glob,sys
    sys.path.append(r'/n/home13/pzheng/Documents/python-functions/python-functions-library')
    from LibraryConstruction import fastaread,fastawrite,fastacombine
    import LibraryDesigner as ld
    import numpy as np
    
    def _check_primer_usage(pb_records=pb_records, fwd_primer=fwd_primer, rev_primer=rev_primer,
                            _RNA=RNA, _RNA_padding=RNA_padding,
                            _verbose=verbose):
        '''Check whether forward or reverse primer are used in all probes'''
        if _verbose:
            print "-- Checking primer usage, total probes:", len(pb_records)
        fwd_len = len(fwd_primer.seq);
        rev_len = len(rev_primer.seq[-20:].reverse_complement());
        
        for record in pb_records:
            if _RNA:
                fp = record.seq[_RNA_padding : _RNA_padding+fwd_len];
                rp = record.seq[-rev_len-_RNA_padding: -RNA_padding];
            else:
                fp = record.seq[:fwd_len].upper();
                rp = record.seq[-rev_len:]
            # checking
            if fp != fwd_primer.seq:
                if _verbose:
                    print "--- Forward primer incorrect in", record
                return False
            if rp != rev_primer.seq[-20:].reverse_complement():
                if _verbose:
                    print "--- Forward primer incorrect in", record
                return False
        return True # if no error applies
    
    def _check_region_size(pb_records, pb_lists=pb_lists, _RNA=RNA):
        '''Generate a dirctionary '''
        # get original region size
        _reg_size_dic = {}
        for lst in pb_lists:
            _reg_size_dic[lst[0]['reg_index']] = len(lst);
        # get region size from probe names
        _size_from_rec = {}
        for record in pb_records:
            if _RNA:
                reg_id = record.id.split('gene_')[1].split('_')[0];
            else:
                reg_id = int(record.id.split('reg_')[1].split('_')[0]);
            if reg_id not in _size_from_rec.keys():
                _size_from_rec[reg_id] = 1; # if not in key, create
            else:
                _size_from_rec[reg_id] += 1; # otherwise, add count
        # compare
        _match = True;
        for k,v in sorted(_size_from_rec.items()):
            if k not in _reg_size_dic.keys():
                print "region list and region id in probes not match for", k
                _match = False
                break
            else:
                if v != _reg_size_dic[k]:
                    print "region size doesn't match for:", k
                    _match = False
                    break
    
        return _reg_size_dic, _match;
    

    def _check_region_to_barcode(pb_records=pb_records, stv_barcodes=stv_barcodes, ndb_barcodes=ndb_barcodes,
                                 total_bc=total_bc, _RNA=RNA):
        '''Generate map from region id to barcodes used in this region'''
        import re
        _reg_to_barcode = {}
        for record in pb_records:
            # region id
            if _RNA:
                reg_id = record.id.split('gene_')[1].split('_')[0];
            else:
                reg_id = int(record.id.split('reg_')[1].split('_')[0]);
            # put reg_id into reg_to_barcode dic
            if reg_id not in _reg_to_barcode.keys():
                # barcode ids
                stv_matches = re.findall('\'STV_(.+?)\'', record.id.upper(), re.DOTALL)
                ndb_matches = re.findall('\'NDB_(.+?)\'', record.id.upper(), re.DOTALL)
                stv_names = ['Stv_'+str(stv_id) for stv_id in stv_matches]
                ndb_names = ['NDB_'+str(ndb_id) for ndb_id in ndb_matches]
                _reg_to_barcode[reg_id] = stv_names+ndb_names
        
        ## barcode check
        _barcode_check = True;
        # barcode names
        bc_names = [stv.id for stv in stv_barcodes] + [ndb.id for ndb in ndb_barcodes]
        # search through previous dictionary
        for reg,bcs in sorted(_reg_to_barcode.items()):
            for bc in bcs:
                if bc not in bc_names:
                    print "-- Wrong barcode name for barcode: "+str(bc)+", region: "+str(reg)
                    _barcode_check = False
                    break
        
        return _reg_to_barcode, _barcode_check;
        
    def _parsing_probe_sequence(record, fwd_primer=fwd_primer, rev_primer=rev_primer,
                                add_rand_gap=add_rand_gap, barcode_len=barcode_len, target_len=target_len,
                                _RNA=RNA, _RNA_padding=RNA_padding):
        '''parse a probe sequence to acquire all barcode binding sites'''
        # take in a seq record, parse the sequence and return a list of all included barcodes (20mer,RC)
        barcode_list = [];
        if RNA:
            _main_seq = record.seq[_RNA_padding+len(fwd_primer.seq):-20-_RNA_padding];
        else:
            _main_seq = record.seq[len(fwd_primer.seq):-20];
        
        
        # trim first 2 barcodes
        for i in range(2):
            barcode_list.append(_main_seq[:barcode_len]);
            _main_seq = _main_seq[(barcode_len+add_rand_gap):];
        # trim all barcodes from the end
        while len(_main_seq) > target_len:
            barcode_list.append(_main_seq[-barcode_len:]);
            _main_seq = _main_seq[:-(barcode_len+add_rand_gap)];
        
        return barcode_list;
    
    def _finding_barcode_name(barcode_list, stv_barcodes=stv_barcodes, ndb_barcodes=ndb_barcodes, 
                              barcode_len=barcode_len, total_bc=total_bc):
        '''Given barcode list generated by parsing probe, return a list of barcode names'''
        _name_list = [];
        for bc_site in barcode_list:
            for bc in stv_barcodes+ndb_barcodes:
                if bc.seq[-barcode_len:] == bc_site.reverse_complement():
                    _name_list.append(bc.id);
                    break;
        
        if len(_name_list) < total_bc:
            print "-- Failed in finding some barcodes."
            print barcode_list, _name_list
        return _name_list;
    
    
    def _check_barcode_to_region(reg_to_barcode, 
                                 pb_records=pb_records, stv_barcodes=stv_barcodes, ndb_barcodes=ndb_barcodes, _RNA=RNA):
        '''Generate map from barcode id to region id'''
        _barcode_to_reg = {}
        _reg_id_exists = []
        for record in pb_records:
            # region id
            if _RNA:
                reg_id = record.id.split('gene_')[1].split('_')[0];
            else:
                reg_id = int(record.id.split('reg_')[1].split('_')[0]);

            if reg_id in _reg_id_exists:
                continue;
            else:
                _barcode_list = _parsing_probe_sequence(record)
                _name_list = _finding_barcode_name(_barcode_list)
                for _n in _name_list:
                    if _n not in _barcode_to_reg.keys(): # create if not in dic
                        _barcode_to_reg[_n] = [reg_id]
                    else: # otherwise, append
                        _barcode_to_reg[_n].append(reg_id)
            _reg_id_exists.append(reg_id)
        ## check region distribution
        # invert dic from reg_to_barcode
        _inv_dic = {}
        for reg,bcs in sorted(reg_to_barcode.items()):
            for bc in bcs:
                if bc not in _inv_dic.keys():
                    _inv_dic[bc] = [reg];
                else:
                    _inv_dic[bc].append(reg);
        # compare
        _region_check=True
        for bc, regs in sorted(_inv_dic.items()):
            if bc not in _barcode_to_reg.keys():
                print "-- "+str(bc)+" not in barcode_to_region dic!"
                print '---',bc
                _region_check = False
                break
            else:
                for reg in regs:
                    if reg not in _barcode_to_reg[bc]:
                        print "-- some barcode in "+str(regs)+" not compatible with "+str(bc)+" in barcode_to_region dic!"
                        _region_check = False
                        break
                for reg in _barcode_to_reg[bc]:
                    if reg not in regs:
                        print "-- some barcode in "+str(bc)+" not compatible with "+str(regs)+" in barcode_to_region dic!"
                        _region_check = False
                        break
                    
        return _barcode_to_reg, _region_check
    
    def _check_barcode_to_color(pb_records=pb_records, stv_barcodes=stv_barcodes, ndb_barcodes=ndb_barcodes, 
                                stv_color=True, ndb_color=False,
                                _save=save, master_dir=master_dir, save_dir=save_dir):
        '''If multi_color is applied, generate a barcode_to_color dic for adaptor design'''
        if 'color' not in str(pb_records[0].id):
            print "-- color check not applied";
            return False
        elif not stv_color and not ndb_color:
            print "-- color check turned off in both stv and ndb";
            return False
        else:
            # get barcodes
            _barcode_names = []
            if stv_color: # if stv has multi-color
                _barcode_names += [bc.id for bc in stv_barcodes];
            if ndb_color: # if ndb has multi-color
                _barcode_names += [bc.id for bc in ndb_barcodes];
            # initialize color dic
            _barcode_to_color = {};
            _exist_regs = [];
            # search through all probes
            for record in pb_records:
                # reg_id
                if _RNA:
                    reg_id = record.id.split('gene_')[1].split('_')[0];
                else:
                    reg_id = int(record.id.split('reg_')[1].split('_')[0]);
                
                if _reg_id in _exist_regs:
                    continue
                else: 
                    _exist_regs.append(_reg_id);
                _color = int(str(record.id).split('color_')[1])
                _barcode_list = _parsing_probe_sequence(record)
                _name_list = _finding_barcode_name(_barcode_list)
                
                for _name in _name_list:
                    if _name in _barcode_names:
                        if _name not in _barcode_to_color.keys():
                            _barcode_to_color[_name] = [_color]
                        else:
                            _barcode_to_color[_name].append(_color);
            # keep the unique colors
            _barcode_to_unique_color = {}
            for k,v in sorted(_barcode_to_color.items()):
                _barcode_to_unique_color[k] = np.unique(v)
            if _save:
                import csv
                # mkdir if not exist for this region
                if not os.path.exists(master_dir+os.sep+save_dir):
                    os.makedirs(master_dir+os.sep+save_dir)
                with open(master_dir+os.sep+save_dir+os.sep+'color-usage.csv','w') as output_handle:
                    fieldnames = ['barcode', 'color']
                    writer = csv.DictWriter(output_handle, fieldnames=fieldnames)
                    writer.writeheader()
                    for _barcode, _color in sorted(_barcode_to_unique_color.items(), key=lambda (k,v):int(k.split('_')[1])):
                        writer.writerow({'barcode': _barcode, 'color': _color})
                
        return _barcode_to_unique_color
                            
    
    def _construct_internal_map(master_dir=master_dir, save_dir=save_dir, word_size=word_size):
        '''Using functions in LibraryDesign, compute an internal khmer map'''
        _int_map = khmer.Countgraph(word_size, 1e9, 2) 
        _int_map.set_use_bigcount(True)
        _nms,_seqs = fastaread(master_dir+os.sep+save_dir+os.sep+'candidate_probes.fasta')
        for _seq in _seqs:
            _int_map.consume(_seq.upper())
        return _int_map
    
    def _check_barcode_in_probes(barcode_to_reg, reg_size_dic, int_map, 
                                 stv_barcodes=stv_barcodes, ndb_barcodes=ndb_barcodes,
                                 barcode_len=barcode_len, max_internal_hits=max_internal_hits):
        '''Check barcode appearance in probes, whether that match barcode_to_region scheme'''
        _barcode_in_probes = {}
        for bc_name, regs in sorted(barcode_to_reg.items()):
            bc = None
            for _bc in stv_barcodes+ndb_barcodes:
                if bc_name == _bc.id:
                    bc = _bc
                    break
            bc_hits = int_map.get_kmer_counts( str(bc.seq[-barcode_len:].reverse_complement()).upper());
            if max(bc_hits) - min(bc_hits) > max_internal_hits:
                print "-- Barcode: "+str(bc)+" has more off-target in different part of itself!"
                return False
            else:
                regs,reg_cts = np.unique(regs, return_counts=True);
                bc_in_probe = 0;
                for reg,ct in zip(regs,reg_cts):
                    bc_in_probe += reg_size_dic[reg] * ct;
                if max(bc_hits) - bc_in_probe > max_internal_hits:
                    print "-- Barcode: "+str(bc)+" has more off-target than threshold!"
                    return False
            _barcode_in_probes[bc_name] = bc_in_probe;
        return _barcode_in_probes, True
    
    def _check_between_probes(int_map, pb_lists=pb_lists, pb_records=pb_records):
        pass 
    
    def _check_against_genome(pb_records=pb_records, max_genome_hits=max_genome_hits, index_folder=index_folder):
        '''Use Khmer to compare probe against genome'''
        hg38 = khmer.load_countgraph(index_folder+os.sep+'full_word17_.kmer')
        _failed_num = 0;
        _keep_pb_records = [];
        for record in pb_records:
            _kmer_hits = hg38.get_kmer_counts(str(record.seq).upper());
            if sum(_kmer_hits) > max_genome_hits:
                print '-- Max_genome_hits is: '+str(max_genome_hits)+", this seq got hits: "+ str(sum(_kmer_hits))
                _failed_num += 1;
            else:
                _keep_pb_records.append(record);
                
        return _keep_pb_records, _failed_num # if nothing goes wrong
    
    def _plot_info():
        pass
            
    ## check primers
    primer_usage = _check_primer_usage()
    if verbose:
        print "\n- 1.Passing primer usage check? -", primer_usage
    
    ## check region size
    reg_size_dic, size_match = _check_region_size(pb_records)
    if verbose:
        print "\n- 2.Passing region size check? -", size_match    
        for k,v in sorted(reg_size_dic.items()):
            print k,':',v
        
    ## check region to barcode
    reg_to_barcode, reg2bc = _check_region_to_barcode()
    if verbose:
        print "\n- 3.Passing region to barcode mapping check? -", reg2bc    
        for k,v in sorted(reg_to_barcode.items(), key=lambda (k,v):k):
            print k,':',v
        
    ## check barcode to region (this step must be run after step 3) 
    barcode_to_reg, bc2reg = _check_barcode_to_region(reg_to_barcode)
    if verbose:
        print "\n- 4.Passing barcode to region mapping check? -", bc2reg    
        for k,v in sorted(barcode_to_reg.items(), key=lambda (k,v):[k[0],int(k.split('_')[1])]):
            print k,':',v
    
    ## check barcode to region (this step must be run after step 3) 
    barcode_to_color = _check_barcode_to_color()
    if verbose and barcode_to_color:
        print "\n- 5.Calculating barcode to color dictionary."
        for k,v in sorted(barcode_to_color.items(), key=lambda (k,v):[k[0],int(k.split('_')[1])]):
            print k,':',v    
    
    
    ## Construct an internal map
    int_map = _construct_internal_map();
    if verbose:
        print "\n- 6.Constructing internal khmer map";
    
    ## Check barcodes total counts in probes
    barcode_in_probes, _bc_counting = _check_barcode_in_probes(barcode_to_reg, reg_size_dic, int_map)
    if verbose:
        print "\n- 7.Passing if counting barcode appearance times in probes", _bc_counting;    

    ## Check against each other    
    
    ## Check against genome
    kept_records, failed_num = _check_against_genome();
    if verbose:
        print "\n- 8.Probes not passing through genome filter:", failed_num;  
    
    # check region size for kept probes
    kept_size_dic, kept_match = _check_region_size(kept_records);
    if verbose:
        print "\n- 9.Re-check region size:"
        for k,v in sorted(kept_size_dic.items()):
            print k,':',v
        print "--- total number of probes:", len(kept_records);
        
    if save:
        pb_savefile = master_dir + os.sep + save_dir + os.sep + 'filtered_probes.fasta';
        if verbose:
            print "\n- 10.Saving probes to:", pb_savefile
        with open(pb_savefile, 'w') as output_handle:
            SeqIO.write(kept_records, output_handle, 'fasta');  
        
    return kept_records, kept_size_dic

In [6]:
# master directory
master_dir = r'/n/boslfs/LABS/zhuang_lab/User/pzheng/Libraries/CTP-02/EMT_smFISH';

kept_records, kept_size_dic = Check_Probes(pb_records, pb_lists, master_dir, target_len=30,
                                           RNA=True,RNA_padding=6,
                                        fwd_primer=fprimer, rev_primer=rprimer,
                                        stv_barcodes=stv_barcodes, ndb_barcodes=ndb_barcodes)

-- Checking primer usage, total probes: 2095

- 1.Passing primer usage check? - True

- 2.Passing region size check? - True
AlphaCat : 56
AlphaVIntegrin : 90
BLACAT1 : 47
BetaCat : 35
CD133 : 47
CD24 : 19
CD44 : 114
CDH1 : 13
CDH2 : 47
CDH3 : 79
DDR2 : 57
ERK2 : 60
ESRP1 : 54
ESRP2 : 68
FN1 : 101
GRHL2 : 24
HOTAIR : 36
KRT14 : 15
KRT18 : 41
MALAT1 : 88
MMP2 : 65
MMP9 : 45
NEAT1 : 98
OCLN : 75
OVOL2 : 32
RBFOX2 : 95
SMAD4 : 100
SNAI1 : 34
SNAI2 : 34
SNAI3 : 26
SRSF1 : 27
TCF3 : 26
TWIST1 : 17
VIM : 79
ZEB1 : 49
ZEB2 : 143
ZO1 : 59

- 3.Passing region to barcode mapping check? - True
AlphaCat : ['NDB_301']
AlphaVIntegrin : ['NDB_302']
BLACAT1 : ['NDB_303']
BetaCat : ['NDB_304']
CD133 : ['NDB_305']
CD24 : ['NDB_306']
CD44 : ['NDB_307']
CDH1 : ['NDB_308']
CDH2 : ['NDB_309']
CDH3 : ['NDB_310']
DDR2 : ['NDB_311']
ERK2 : ['NDB_312']
ESRP1 : ['NDB_313']
ESRP2 : ['NDB_314']
FN1 : ['NDB_315']
GRHL2 : ['NDB_316']
HOTAIR : ['NDB_317']
KRT14 : ['NDB_318']
KRT18 : ['NDB_319']
MALAT1 : ['NDB_320']
MM